<a href="https://colab.research.google.com/github/tztechno/cc_archive/blob/main/ASL_Alphabet_Classification_w_TensorFlow_js_Export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
grassknoted_asl_alphabet_path = kagglehub.dataset_download('grassknoted/asl-alphabet')

print('Data source import complete.')


# **ASL Alphabet Classification w/TensorFlow.js Export**

In [ ]:
!pip install tensorflow==2.15.0
!pip install protobuf==3.20.3

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
from pathlib import Path
import random

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

print("Loading data from folders...")

# Set your data path here
data_path = '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train'

# Get all image paths and labels
image_paths = []
labels = []

# Create a mapping from folder names to class indices
class_folders = sorted([f for f in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, f))])
class_to_index = {class_name: idx for idx, class_name in enumerate(class_folders)}

print(f"Found classes: {class_folders}")
print(f"Number of classes: {len(class_folders)}")

# Iterate through each class folder
for class_folder in class_folders:
    class_path = os.path.join(data_path, class_folder)
    label = class_to_index[class_folder]

    # Get all image files in this class
    class_images = [f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Sample 2000 images per class (increased from 600)
    sampled_images = random.sample(class_images, min(1000, len(class_images)))

    for img_file in sampled_images:
        image_paths.append(os.path.join(class_path, img_file))
        labels.append(label)

print(f"Found {len(image_paths)} images across {len(set(labels))} classes")

# Load and preprocess images
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    """Load image and convert to normalized RGB array"""
    img = keras.preprocessing.image.load_img(
        image_path,
        color_mode='rgb',
        target_size=target_size
    )
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = img_array / 255.0  # Normalize to [0, 1]
    return img_array

# Load all images in batches to avoid memory issues
print("Loading and preprocessing images...")
batch_size = 100
X = []
for i in range(0, len(image_paths), batch_size):
    batch_paths = image_paths[i:i+batch_size]
    batch_images = [load_and_preprocess_image(path) for path in batch_paths]
    X.extend(batch_images)
    print(f"  Loaded {min(i+batch_size, len(image_paths))}/{len(image_paths)} images...")

X = np.array(X)
y = np.array(labels)
print("All images loaded!")

# Convert labels to categorical
num_classes = len(set(labels))
y = keras.utils.to_categorical(y, num_classes)

print(f"\nData shape: {X.shape}")
print(f"Labels shape: {y.shape}")

# Split into training and validation data
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.15, random_state=42
)

print(f"Training Data: {X_train.shape}")
print(f"Validation Data: {X_val.shape}")

# Build a simplified and more efficient model
print("\nBuilding model...")
model = keras.Sequential([
    # Input layer
    keras.layers.InputLayer(input_shape=(224, 224, 3)),

    # First Convolutional Block
    keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.2),

    # Second Convolutional Block
    keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.2),

    # Third Convolutional Block
    keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.3),

    # Fourth Convolutional Block
    keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.3),

    # Fully Connected Layers
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model with lower learning rate
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),  # Lower learning rate
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Display model summary
model.summary()

# Data augmentation (REMOVED horizontal_flip - critical for sign language!)
print("\nConfiguring data augmentation...")
datagen = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=10,           # Reduced from 15
    zoom_range=0.1,              # Reduced from 0.15
    width_shift_range=0.1,       # Reduced from 0.15
    height_shift_range=0.1,      # Reduced from 0.15
    brightness_range=[0.9, 1.1], # Narrowed range
    fill_mode='nearest'
)

# Configure callbacks
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,  # Increased patience
        restore_best_weights=True,
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,  # Increased patience
        min_lr=0.000001,
        verbose=1
    ),
    keras.callbacks.ModelCheckpoint(
        'best_asl_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

# Train the model
print("\nTraining the model...")
print(f"Steps per epoch: {len(X_train) // 128}")
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=128),  # Increased batch size
    steps_per_epoch=len(X_train) // 128,
    epochs=50,  # Increased epochs
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1
)

# Save the final model
print("\nSaving the model...")
model.save('final_asl_model.h5')
print("✓ Model saved as 'final_asl_model.h5'")

# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot accuracy
ax1.plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
ax1.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
ax1.set_title('Model Accuracy', fontsize=14, fontweight='bold')
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Accuracy', fontsize=12)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Plot loss
ax2.plot(history.history['loss'], label='Training Loss', linewidth=2)
ax2.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
ax2.set_title('Model Loss', fontsize=14, fontweight='bold')
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Loss', fontsize=12)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_history.png', dpi=300)
print("✓ Training history saved as 'training_history.png'")

# Evaluate on validation set
print("\nEvaluating model...")
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
print(f"\nFinal Validation Accuracy: {val_accuracy:.4f} ({val_accuracy*100:.2f}%)")
print(f"Final Validation Loss: {val_loss:.4f}")

# Display class mapping for reference
print("\n" + "="*60)
print("Class Mapping:")
print("="*60)
for class_name, idx in sorted(class_to_index.items(), key=lambda x: x[1]):
    print(f"{idx:2d}: {class_name}")

# Display command for converting to TensorFlow.js format
print("\n" + "="*60)
print("Next Step: Convert to TensorFlow.js format")
print("="*60)
print("\nRun the following commands:")
print("\n  pip install tensorflowjs")
print("\n  tensorflowjs_converter --input_format=keras \\")
print("      final_asl_model.h5 \\")
print("      ./tfjs_model/")
print("\nAfter conversion, place the files in './tfjs_model/' folder")
print("into the same directory as your web application.")
print("="*60)

# Summary of improvements made
print("\n" + "="*60)
print("KEY IMPROVEMENTS IN THIS VERSION:")
print("="*60)
print("1. ✓ Removed horizontal_flip (critical for sign language)")
print("2. ✓ Lowered learning rate from 0.001 to 0.0001")
print("3. ✓ Increased data per class from 600 to 2000 images")
print("4. ✓ Simplified model architecture (reduced parameters)")
print("5. ✓ Reduced augmentation intensity")
print("6. ✓ Increased batch size from 64 to 128")
print("7. ✓ Increased patience for callbacks")
print("8. ✓ Set random seeds for reproducibility")
print("="*60)

## [**ASL Sign Language Recognition Web App**](https://tztechno.github.io/tz_tensorflowjs/03_ASL_Alphabet/index.html)